In [132]:
import json 
import os
import logging 
import concurrent 
from datetime import date, datetime 

from typing import List 
from pprint import PrettyPrinter
from subgrounds import Subgrounds
from subgrounds.pagination import ShallowStrategy
from IPython.display import HTML, display
from web3 import Web3 
from functools import partial 
from concurrent.futures import ThreadPoolExecutor
import pandas as pd 
import numpy as np 
import altair as alt 
from python_graphql_client import GraphqlClient


# logging.basicConfig(level=logging.DEBUG) 

pp = PrettyPrinter().pprint

In [2]:
URL_INFURA = 'https://mainnet.infura.io/v3/856c3834f317452a82e25bb06e04de18'
w3 = Web3(Web3.HTTPProvider(URL_INFURA))

### Bribes Analysis 

We are interested in analyzing the impact of bribes through 

- Votium (voting power vlCVX, incentives in FXS) 
- Pitch 
- bribe.crv.finance 
- Hidden Hand 
- Aura 

that incentivize liquidity in liquidity pools that contain FRAX. To start, we are working with the following subset of pools that exist on curve. 
- Metapools (stable / non-stable). 
- Frax BP
- Frax:3crv

The following metrics are of interest

- \\$ Bribes spent per pool (broken down by platform and incentive type)  
- \\$ Liquidity per pool (broken into Frax and non-Frax components) 

Once we have both the liquidity and bribe information, we can try to answer these questions
- Where are bribe dollars best spent? 
- How does the Frax protocol optimize its bribing strategy in order to maximize the amount of liquidity within it's pools? 

In [3]:
def ddf(df):
    display(HTML(df.to_html()))
    
    
def remove_prefix(df: pd.DataFrame, prefix: str):
    # Remove a prefix from all columns 
    cols = [c for c in df.columns]
    for i, c in enumerate(cols): 
        if c.startswith(prefix): 
            cols[i] = c[len(prefix):]
    df.columns = cols 
    return df 

def remove_prefixes(df: pd.DataFrame, prefixes: List[str]):
    for p in prefixes: 
        df = remove_prefix(df, p)
    return df 

def zip_dfs(dfs, col_names): 
    data = dfs[0]
    for i, c in enumerate(col_names): 
        data = data.merge(dfs[i+1][[c]], how='left', left_index=True, right_index=True)
    return data

def query_attrs(query, attrs):
    return sg.query_df([getattr(query, a) for a in attrs], pagination_strategy=ShallowStrategy) 

In [4]:
# Non-subgraph
URL_SNAPSHOT = 'https://hub.snapshot.org/graphql'
# Subgraph 
URL_VOTIUM = 'https://api.thegraph.com/subgraphs/name/convex-community/votium-bribes' 
URL_CURVE_POOLS = 'https://api.thegraph.com/subgraphs/name/convex-community/volume-mainnet' 

# Addresses 
VOTIUM_VOTER = '0xde1E6A7ED0ad3F61D531a8a78E83CcDdbd6E0c49'.lower()
CURVE_POOL_FRAX_USDC = '0xdcef968d416a41cdac0ed8702fac8128a64241a2'

In [5]:
# Instantiate the client with an endpoint.
client = GraphqlClient(endpoint=URL_SNAPSHOT)
sg = Subgrounds()

# votium_bribes = sg.load_subgraph(URL_VOTIUM) 
sg_curve = sg.load_subgraph(URL_CURVE_POOLS)
sg_votium = sg.load_subgraph(URL_VOTIUM) 

### Metapool TVL  

In [6]:
def get_curve_metapools():
    """Queries the curve volume subgraph `Pools` entity to get data for all curve metapools 
    
    """
    # The name, type, and address of some of the largest curve metapools 
    curve_metapools = [
        ('agEUR+FRAXBP', 'CRYPTO V2', '0x58257e4291f95165184b4bea7793a1d6f8e7b627'),
        ('ALCX+FRAXBP', 'CRYPTO V2', '0x4149d1038575ce235e03e03b39487a80fd709d31'),
        ('alUSD+FRAXBP', 'USD', '0xB30dA2376F63De30b42dC055C93fa474F31330A5'),
        ('ApeUSD+FRAXBP', 'USD', '0x04b727C7e246CA70d496ecF52E6b6280f3c8077D'),
        ('BADGER+FRAXBP', 'CRYPTO V2', '0x13b876c26ad6d21cb87ae459eaf6d7a1b788a113'),
        ('BENT+FRAXBP', 'CRYPTO V2', '0x825722af244432319c1e32b6b18aded2d4a014df'),
        ('BUSD+FRAXBP', 'USD', '0x8fdb0bB9365a46B145Db80D0B1C5C5e979C84190'),
        ('CVX+FRAXBP', 'CRYPTO V2', '0xbec570d92afb7ffc553bdd9d4b4638121000b10d'),
        ('cvxCRV+FRAXBP', 'CRYPTO V2', '0x31c325a01861c7dbd331a9270296a31296d797a0'),
        ('cvxFXS+FRAXBP', 'CRYPTO V2', '0x21d158d95c2e150e144c36fc64e3653b8d6c6267'),
        ('DOLA+FRAXBP', 'USD', '0xE57180685E3348589E9521aa53Af0BCD497E884d'),
        ('GUSD+FRAXBP', 'USD', '0x4e43151b78b5fbb16298C1161fcbF7531d5F8D93'),
        ('LUSD+FRAXBP', 'USD', '0x497CE58F34605B9944E6b15EcafE6b001206fd25'),
        ('MAI+FRAXBP', 'USD', '0x66E335622ad7a6C9c72c98dbfCCE684996a20Ef9'),
        ('pUSD+FRAXBP', 'USD', '0xC47EBd6c0f68fD5963005D28D0ba533750E5C11B'),
        ('RSR+FRAXBP', 'CRYPTO V2', '0x6a6283ab6e31c2aec3fa08697a8f806b740660b2'),
        ('SDT+FRAXBP', 'CRYPTO V2', '0x3e3c6c7db23cddef80b694679aaf1bcd9517d0ae'),
        ('sUSD+FRAXBP', 'USD', '0xe3c190c57b5959Ae62EfE3B6797058B76bA2f5eF'),
        ('TUSD+FRAXBP', 'USD', '0x33baeDa08b8afACc4d3d07cf31d49FC1F1f3E893'),
        ('USDD+FRAXBP', 'USD', '0x4606326b4Db89373F5377C316d3b0F6e55Bc6A20'),
        ('XAI+FRAXBP', 'USD', '0x326290a1b0004eee78fa6ed4f1d8f4b2523ab669'),
    ] 
    pool_addresses = [e[2] for e in curve_metapools] 
    pool_address_type_map = {e[2].lower(): e[1] for e in curve_metapools}
    pool_ids = list(set(pool_addresses + [e.lower() for e in pool_addresses])) # normalize for lower case 
    
    # query pool data 
    query_pools = sg_curve.Query.pools(first=50, where={"id_in": pool_ids})
    dfs_pools = query_attrs(query_pools, ['id', 'name', 'lpToken', 'symbol', 'coins', 'coinNames', 'coinDecimals'])
    
    # process pool data
    df_pools = zip_dfs(dfs_pools, ['pools_coinNames', 'pools_coinDecimals'])
    df_pools = df_pools.reset_index() 
    df_pools['coin_index'] = (
        # There's one row in this df for each combination of a pool and a coin. So a pool with 3 coins will lead to 3 rows. 
        # When subgrounds flattens data in producing output dataframes, the order of the rows is preserved so this method 
        # is fine for determining the coin index (assuming we didn't do any sorting after pulling the data. 
        df_pools.groupby('pools_id', sort=False)['index'].rank(method='first').astype(int)
    )
    df_pools['pool_type'] = df_pools.pools_id.apply(lambda a: pool_address_type_map[a.lower()])
    df_pools = df_pools.drop(columns=['index'])
    assert len(curve_metapools) == len(df_pools.pools_symbol.unique()) 
    df_pools = df_pools.rename(columns={'pools_id': 'pool_address'})
    
    return df_pools

    
def get_curve_pool_reserves(pool_ids, verbose=True): 
    # Query pool snapshots 
    nrows = -1
    nreturned = 1000
    while nrows == -1 or nrows == nreturned:
        nreturned *= 10
        query_pool_snaps = sg_curve.Query.dailyPoolSnapshots(where={"pool_in": pool_ids}, first=nreturned)
        df_reserves = query_attrs(query_pool_snaps, ['id', 'timestamp', 'reservesUSD']).reset_index()
        nrows = len(df_reserves)

    # Process pool snapshots 
    df_reserves['timestamp'] = pd.to_datetime(df_reserves.dailyPoolSnapshots_timestamp, unit='s')
    df_reserves['pool_address'] = df_reserves.dailyPoolSnapshots_id.apply(lambda _id: _id.split('-')[0])
    df_reserves['coin_index'] = (
        df_reserves.groupby(['dailyPoolSnapshots_id', 'dailyPoolSnapshots_timestamp'], sort=False)['index'].rank(method='first').astype(int)
    )
    df_reserves = df_reserves.drop(columns=['index'])
    return df_reserves

In [7]:
df_pools = get_curve_metapools()

In [8]:
df_pools.head()

,pool_address,pools_name,pools_lpToken,pools_symbol,pools_coins,pools_coinNames,pools_coinDecimals,coin_index,pool_type
0,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,Curve.fi Factory USD Metapool: apeUSDFRAXBP,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,APEUSDBP3CRV-f,0xff709449528b6fb6b88f557f7d93dece33bca78d,ApeUSD,18,1,USD
1,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,Curve.fi Factory USD Metapool: apeUSDFRAXBP,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,APEUSDBP3CRV-f,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,crvFRAX,18,2,USD
2,0x13b876c26ad6d21cb87ae459eaf6d7a1b788a113,Curve.fi Factory Crypto Pool: BADGER/FRAXBP,0x09b2e090531228d1b8e3d948c73b990cb6e60720,BADGERFRAX-f,0x3472a5a71965499acd81997a54bba8d852c6e53d,BADGER,18,1,CRYPTO V2
3,0x13b876c26ad6d21cb87ae459eaf6d7a1b788a113,Curve.fi Factory Crypto Pool: BADGER/FRAXBP,0x09b2e090531228d1b8e3d948c73b990cb6e60720,BADGERFRAX-f,0x3175df0976dfa876431c2e9ee6bc45b65d3473cc,crvFRAX,18,2,CRYPTO V2
4,0x21d158d95c2e150e144c36fc64e3653b8d6c6267,Curve.fi Factory Crypto Pool: cvxFxs/FraxBP,0xf57ccad8122b898a147cc8601b1eca88b1662c7e,cvxFxsFrax-f,0xfeef77d3f69374f66429c91d732a244f074bdf74,cvxFXS,18,1,CRYPTO V2


In [10]:
pool_ids = df_pools.pool_address.unique().tolist()
df_reserves = get_curve_pool_reserves(df_pools.pool_address.unique().tolist())

In [11]:
df_reserves.head()

,dailyPoolSnapshots_id,dailyPoolSnapshots_timestamp,dailyPoolSnapshots_reservesUSD,timestamp,pool_address,coin_index
0,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d-165...,1656547200,0.000000,2022-06-30,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,1
1,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d-165...,1656547200,0.000000,2022-06-30,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,2
2,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d-165...,1656633600,0.000000,2022-07-01,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,1
3,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d-165...,1656633600,0.000000,2022-07-01,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,2
4,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d-165...,1656720000,66782.220309,2022-07-02,0x04b727c7e246ca70d496ecf52e6b6280f3c8077d,1


In [12]:
df_liquidity = df_reserves.merge(
    df_pools, how='left', on=['pool_address', 'coin_index'], validate="m:1" 
)

In [13]:
# df_liquidity.loc[df_liquidity.pools_symbol == 'BADGERFRAX-f'].tail()

### Metapool TVL Charts 

#### TVL Faceted by Metapool Segmented by Coin Name 

In [14]:
(
    alt.Chart(df_liquidity)
    .mark_area()
    .transform_calculate(stack_order="datum.pools_coinNames === 'crvFRAX' ? 0 : 1")
    .encode(
        x="dailyPoolSnapshots_timestamp:T", 
        y="dailyPoolSnapshots_reservesUSD:Q", 
        color=alt.Color("pools_coinNames:N", scale=alt.Scale(scheme="tableau20")), 
        tooltip=alt.Tooltip("dailyPoolSnapshots_reservesUSD:Q", format='$,d'), 
        facet=alt.Facet('pools_symbol:N', columns=3),
        order="stack_order:O", 
    )
    .resolve_scale(y="independent")
)

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

#### Current Metapool Liquidity Faceted by Asset Type Segmented by Metapool

In [17]:
df_tvl = (
    df_liquidity[df_liquidity.pools_coinNames != 'crvFRAX']
    .groupby("pools_name", sort="timestamp").last().reset_index()
    [['dailyPoolSnapshots_reservesUSD', 'pool_type', 'pools_coinNames']]
)
df_tvl = df_tvl.merge(
    df_tvl.groupby('pool_type')['dailyPoolSnapshots_reservesUSD'].sum().reset_index().rename(columns={'dailyPoolSnapshots_reservesUSD': 'tvl_total'}), 
    how='left', on="pool_type", validate="m:1"
)
df_tvl['tvl_share'] = df_tvl.dailyPoolSnapshots_reservesUSD / df_tvl.tvl_total
# df_tvl.head()

In [18]:
(
    alt.Chart(df_tvl)
    .mark_arc()
    .encode(
        theta="dailyPoolSnapshots_reservesUSD:Q", 
        color=alt.Color("pools_coinNames:N", scale=alt.Scale(scheme="tableau20")), 
        facet=alt.Facet('pool_type:N', columns=3),
        tooltip=[
            alt.Tooltip("pools_coinNames:N", title="Token"), 
            alt.Tooltip("dailyPoolSnapshots_reservesUSD:Q", format="$,d", title="TVL"),
            alt.Tooltip("tvl_share:Q", format=".1%", title="TVL Share")
        ] 
    )
    .resolve_scale(theta="independent")
)

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

#### Current Metapool Liquidity Segmented By Asset Type 

In [19]:
df_tvl_type = df_liquidity.copy()
df_tvl_type.dailyPoolSnapshots_reservesUSD = df_tvl_type.dailyPoolSnapshots_reservesUSD.astype(np.float64)
df_tvl_type = (
    df_tvl_type[df_tvl_type.pools_coinNames != 'crvFRAX']
    .groupby(["pool_type", "timestamp"], dropna=False)['dailyPoolSnapshots_reservesUSD'].sum().reset_index()
    .rename(columns={'dailyPoolSnapshots_reservesUSD': 'tvl_type'})
)
df_tvl_total = df_tvl_type.groupby("timestamp")['tvl_type'].sum().reset_index().rename(columns={'tvl_type': 'tvl_total'})

In [20]:
(
    alt.Chart(df_tvl_type)
    .mark_area()
    .encode(
        x="timestamp:T", 
        y="tvl_type:Q", 
        color="pool_type:N", 
        tooltip=["pool_type:N", alt.Tooltip("tvl_type:Q", format="$,d")]
    ) + 
    alt.Chart(df_tvl_total)
    .mark_line(color="green")
    .encode(
        x="timestamp:T", 
        y="tvl_total:Q", 
        tooltip=[alt.Tooltip("tvl_total:Q", format="$,d")]
    )
)

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

### Snapshot Proposals 

Here, we get the snapshot proposals that correspond to votium votes. Each proposal contains the possible set of choices (liquidity pools that rewards can be directed to). 

In [ ]:
def get_proposals(): 

proposal_attrs = [
    'id',
    'title',
    # 'body',
    'choices',
    'start',
    'end',
    'snapshot',
    'state',
]

query_proposals = """
    query Proposals {
      proposals(
        first: 10000,
        where: { space: "cvx.eth" },
        orderBy: "created",
        orderDirection: desc
      ) {
        <proposal_attrs>
      }
    }
""".replace('<proposal_attrs>', '\n'.join(proposal_attrs))
data_proposals = client.execute(query=query_proposals)['data']['proposals']
# Number of rounds here should match number of rounds on llama airforce 
# https://llama.airforce/#/bribes/rounds/votium/cvx-crv/
data_proposals = [d for d in data_proposals if d['title'].startswith('Gauge Weight for Week')]

In [38]:
# Get all votium voting epochs. Once we have validated that this set of epochs matches our set of snapshot proposals, 
# we need to merge this data with our snapshot proposal data. 
epoches = sg_votium.Query.epoches(first=1000, orderBy="initiatedAt", where={"bribeCount_gt": 0})
df_epoches = sg.query_df([epoches.id, epoches.initiatedAt], pagination_strategy=ShallowStrategy)
# df_epoches = remove_prefix(df_epoches, "epoches_")
df_epoches.epoches_initiatedAt = pd.to_datetime(df_epoches.epoches_initiatedAt, unit="s")
df_epoches['date'] = df_epoches.epoches_initiatedAt.dt.date
start_rank = [pd.Timestamp(d) for d in sorted(np.unique(df_epoches.epoches_initiatedAt.values).tolist())]
df_epoches['round'] = df_epoches.epoches_initiatedAt.apply(lambda d: start_rank.index(d) + 1) 
# Validate that the epoch dates for votium bribes match the proposal data we pulled from snapshot. `
epoch_dates = df_epoches.date.unique().tolist()
proposal_dates = df.start.dt.date.unique().tolist()
d_exclude = pd.Timestamp('2021-11-08').date()
assert d_exclude in epoch_dates and not d_exclude in proposal_dates
epoch_dates.remove(d_exclude)
assert set(epoch_dates) == set(proposal_dates)
df_epoches.head()

,epoches_id,epoches_initiatedAt,date,round
0,0xc841db892a58168d21262eb8e2f97d651fb354896fa9...,2021-09-16 00:04:43,2021-09-16,1
1,0xe11de27c199ddee0966ed8b24e1ee34bd3363f0fe09f...,2021-09-30 00:02:12,2021-09-30,2
2,0x20b26be17c62c2d155f9f9a9a3dac7bc3fcaa4845322...,2021-10-14 00:04:43,2021-10-14,3
3,0xb439c625f9f4929f6b486f79d5ae6850971d179a0887...,2021-10-28 11:02:59,2021-10-28,4
4,0xb59c1e06f38e5daaaa51e672174f6a4a65cf654d1e36...,2021-11-08 15:32:47,2021-11-08,5


In [64]:
# Get proposal data from snapshot. We determine which proposals are for votium using the title. 
# The proposal also contains the possible options voters are choosing between. 

'https://hub.snapshot.org/graphql'

In [125]:
def get_proposals(): 
    proposal_attrs = ['id', 'title', 'choices', 'start', 'end', 'snapshot', 'state']
    query_proposals = """
        query Proposals {
          proposals(
            first: 10000,
            where: {
              space: "cvx.eth"         
              # id: "0xee37337fd2b8b5112ac4efd2948d58e4e44f59ee904c70650d26ece60276ed9f"
            },
            orderBy: "created",
            orderDirection: desc
          ) {
            <proposal_attrs>
          }
        }
    """.replace('<proposal_attrs>', '\n'.join(proposal_attrs))
    proposals = client.execute(query=query_proposals)['data']['proposals']
    # Number of rounds here should match number of rounds on llama airforce 
    # https://llama.airforce/#/bribes/rounds/votium/cvx-crv/
    df_proposals = pd.DataFrame([
        d for d in proposals 
        if d['title'].startswith('Gauge Weight for Week') and d['state'] == 'closed' # TODO: Should we consider active proposals in some way? 
    ])
    df_proposals.start = pd.to_datetime(df_proposals.start, unit='s').dt.date
    df_proposals.end = pd.to_datetime(df_proposals.end, unit='s').dt.date 
    df_proposals = df_proposals.sort_values('start').reset_index(drop=True)
    # On votium, the proposal id is keccak256 hashed. So we need to perform this operation on the data pulled from snapshot 
    df_proposals['id_keccak256'] = df_proposals.id.apply(
        lambda _id: Web3.keccak(text=_id).hex() if not _id.startswith('0x') else Web3.keccak(hexstr=_id).hex()
    )
    # Validate that this query pulls in all data by ensuring that the number of votium voting rounds we get from this query 
    # matches the expected number of voting rounds, which we can compute using simple timedelta logic, the date of the 
    # first vote, and the current date. 
    dmin = pd.Timestamp('2021-09-16') # day of the first votium snapshot 
    assert pd.Timestamp(df_proposals.start.min()) == dmin
    df_proposals['start_diff'] = df_proposals.start.shift(1) - df_proposals.start
    assert (df_proposals.iloc[1:,].start_diff == pd.Timedelta('-14 days')).all()
    # Since a proposal happens every two weeks, and we know the date of the first proposal, we can compute the number of expected proposals on any given date. 
    num_proposals_expected = (
        int(np.ceil((pd.Timestamp(pd.Timestamp.now().date()) - dmin) / pd.Timedelta("14 days")))
    )
    assert len(df_proposals) == num_proposals_expected
    return df_proposals 
    
    
def get_proposal_votes(proposal_ids, verbose=True): 
    """Returns a mapping of proposal ids to votes 
    
    We are pulling proposal information from the snapshot graphql endpoint. 
    """
    with Thread
    
    
    # maps proposal id to votium's vote in that proposal
    vote_map = {}
    for i, pid in enumerate(proposal_ids): 
        if verbose: 
            print(f"Querying votium snapshot proposal {i+1} / {len(proposal_ids)}")
        # Get votium's vote per this round (votes once per round) 
        query_votes = '''
            query Votes {
              votes (
                first: 10000
                where: {proposal: "<proposal_id>", voter: "<voter_id>"}
              ) {
                id
                voter
                created
                choice
                vp
                vp_by_strategy
                vp_state
              }
            }
        '''.replace('<proposal_id>', pid).replace("<voter_id>", VOTIUM_VOTER)
        votes = client.execute(query=query_votes)['data']['votes']
        assert len(votes) == 1
        vote_map[pid] = votes[0]
    return vote_map 
    

In [131]:
df_proposals = get_proposals()
print(f"Number of votium snapshot proposals: {len(df_proposals)}")
df_proposals.head()

Number of votium snapshot proposals: 31


,id,title,choices,start,end,snapshot,state,id_keccak256,start_diff
0,QmUjELF3ABSV2f5xgQrJgEnZTPb86DAtT6gzoa8RfHUuAK,Gauge Weight for Week of 16th Sep 2021,"[compound, usdt, ypool, busd, susd, pax, ren, ...",2021-09-16,2021-09-21,13233425,closed,0xc841db892a58168d21262eb8e2f97d651fb354896fa9...,NaT
1,QmTQBqsG7dW93xX8zBZnevMa1mbEmDHUx7QabAYyn6mFJi,Gauge Weight for Week of 30th Sep 2021,"[compound, usdt, ypool, busd, susd, pax, ren, ...",2021-09-30,2021-10-05,13323756,closed,0xe11de27c199ddee0966ed8b24e1ee34bd3363f0fe09f...,-14 days
2,QmaS9vd1vJKQNBYX4KWQ3nppsTT3QSL3nkz5ZYSwEJk6hZ,Gauge Weight for Week of 14th Oct 2021,"[compound, usdt, ypool, busd, susd, pax, ren, ...",2021-10-14,2021-10-19,13413053,closed,0x20b26be17c62c2d155f9f9a9a3dac7bc3fcaa4845322...,-14 days
3,QmacSRTG62rnvAyBuNY3cVbCtBHGV8PuGRoL32Dm6MPy5y,Gauge Weight for Week of 28th Oct 2021,"[compound, usdt, ypool, busd, susd, pax, ren, ...",2021-10-28,2021-11-02,13505315,closed,0xb439c625f9f4929f6b486f79d5ae6850971d179a0887...,-14 days
4,QmPSBg5aTPb82sZRqF9ouUQQ5CkbpRaJMdHYUMieN3dpqv,Gauge Weight for Week of 11th Nov 2021,"[compound, usdt, ypool, busd, susd, pax, ren, ...",2021-11-11,2021-11-16,13591595,closed,0x77cebe9bd4bc9836f115774c7387839e77abe3dda613...,-14 days


In [130]:
vote_map = get_proposal_votes(df_proposals.id.unique())

Querying votium snapshot proposal 1 / 31
Querying votium snapshot proposal 2 / 31
Querying votium snapshot proposal 3 / 31
Querying votium snapshot proposal 4 / 31
Querying votium snapshot proposal 5 / 31
Querying votium snapshot proposal 6 / 31
Querying votium snapshot proposal 7 / 31
Querying votium snapshot proposal 8 / 31
Querying votium snapshot proposal 9 / 31
Querying votium snapshot proposal 10 / 31
Querying votium snapshot proposal 11 / 31
Querying votium snapshot proposal 12 / 31
Querying votium snapshot proposal 13 / 31
Querying votium snapshot proposal 14 / 31
Querying votium snapshot proposal 15 / 31
Querying votium snapshot proposal 16 / 31
Querying votium snapshot proposal 17 / 31
Querying votium snapshot proposal 18 / 31
Querying votium snapshot proposal 19 / 31
Querying votium snapshot proposal 20 / 31
Querying votium snapshot proposal 21 / 31
Querying votium snapshot proposal 22 / 31
Querying votium snapshot proposal 23 / 31
Querying votium snapshot proposal 24 / 31
Q

In [50]:
records = []
for p in data_proposals: 
    # votium's vote 
    choices = p['choices']
    vote = vote_map[p['id']]
    for choice_index, percent in vote['choice'].items(): 
        choice_index = int(choice_index) - 1  # Choices have indices starting at 1 in snapshot 
        p_attrs = ['title', 'start', 'end', 'snapshot', 'state', 'id']
        fraction = percent / 100 
        # Strategies are weighted voting and delegation. This particular vote is for users who have delegated their voting power to votium. 
        assert vote['vp_by_strategy'][0] == 0
        assert vote['vp_by_strategy'][1] > 0
        records.append({
            **{a: p[a] for a in p_attrs}, 
            **{
                "choice": p['choices'][choice_index],
                "fraction": fraction, 
                "choice_index": choice_index, 
                "vlCVX_total": vote['vp'], 
                "vlCVX_choice": fraction * vote['vp'], 
                # "vp": vote['vp'], 
                # "vp_by_strategy": vote["vp_by_strategy"], 
                # "vp_state": vote["vp_state"],                 
            }
        })
df_raw = pd.DataFrame(records)

In [51]:
def process_df(df):
    start_rank = np.unique(df.start.values).tolist()
    df['round'] = df.start.apply(lambda start: start_rank.index(start) + 1)
    df.start = pd.to_datetime(df.start, unit='s')
    df.end = pd.to_datetime(df.end, unit='s')
    # df.snapshot = pd.to_datetime(df.snapshot, unit='s')
    df = df.sort_values('round').reset_index(drop=True)
    return df 

df = process_df(df_raw.copy())
ddf(df.head())

,title,start,end,snapshot,state,id,choice,fraction,choice_index,vlCVX_total,vlCVX_choice,round
0,Gauge Weight for Week of 16th Sep 2021,2021-09-16 00:04:20,2021-09-21 00:04:20,13233425,closed,QmUjELF3ABSV2f5xgQrJgEnZTPb86DAtT6gzoa8RfHUuAK,cvxcrv,1.76,41,2.171453e+06,3.821758e+06,1
1,Gauge Weight for Week of 16th Sep 2021,2021-09-16 00:04:20,2021-09-21 00:04:20,13233425,closed,QmUjELF3ABSV2f5xgQrJgEnZTPb86DAtT6gzoa8RfHUuAK,eurs,250.00,22,2.171453e+06,5.428634e+08,1
2,Gauge Weight for Week of 16th Sep 2021,2021-09-16 00:04:20,2021-09-21 00:04:20,13233425,closed,QmUjELF3ABSV2f5xgQrJgEnZTPb86DAtT6gzoa8RfHUuAK,alusd,496.22,36,2.171453e+06,1.077519e+09,1
3,Gauge Weight for Week of 16th Sep 2021,2021-09-16 00:04:20,2021-09-21 00:04:20,13233425,closed,QmUjELF3ABSV2f5xgQrJgEnZTPb86DAtT6gzoa8RfHUuAK,frax,1825.19,32,2.171453e+06,3.963315e+09,1
4,Gauge Weight for Week of 16th Sep 2021,2021-09-16 00:04:20,2021-09-21 00:04:20,13233425,closed,QmUjELF3ABSV2f5xgQrJgEnZTPb86DAtT6gzoa8RfHUuAK,ankreth,293.74,27,2.171453e+06,6.378427e+08,1


In [52]:
# Data quality issues: Voting power for votium should be close to 100 and not exceed 100. Rounds 1 and 2 have some data quality issues that need to be resolved 
# later but we can simply ignore them for now. 
tol = .05 # 5 basis points error tolerance 
df_cumulative_fraction = df.groupby("round")['fraction'].sum().reset_index()
df_cumulative_fraction = df_cumulative_fraction.loc[
    (np.abs(df_cumulative_fraction.fraction - 100) >= tol) & # cumulative voting power isn't close to 100% 
    (df_cumulative_fraction.fraction > 100) # Voting power exceeds 100. Voting power can be well below 100 if delegates vote for themselves so we ignore that 
] 
assert set(df_cumulative_fraction['round'].unique()) == set([1, 2])
(
    alt.Chart(df_cumulative_fraction)
    .mark_bar()
    .encode(x="round:O", y="fraction:Q", tooltip="fraction:Q")
)

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [53]:
df.loc[df['round'] == 27].sort_values('fraction').tail()

,title,start,end,snapshot,state,id,choice,fraction,choice_index,vlCVX_total,vlCVX_choice,round
344,Gauge Weight for Week of 15th Sep 2022,2022-09-15 00:00:38,2022-09-20 00:00:38,15535777,closed,bafkreihoebae4qsky5c4da7nljigggd3ytv5pucozwakp...,WETH+CVX (0xB576…),6.06,58,1.858283e+07,1.126120e+08,27
364,Gauge Weight for Week of 15th Sep 2022,2022-09-15 00:00:38,2022-09-20 00:00:38,15535777,closed,bafkreihoebae4qsky5c4da7nljigggd3ytv5pucozwakp...,ApeUSD+crvFRAX (0x04b7…),6.88,102,1.858283e+07,1.278499e+08,27
341,Gauge Weight for Week of 15th Sep 2022,2022-09-15 00:00:38,2022-09-20 00:00:38,15535777,closed,bafkreihoebae4qsky5c4da7nljigggd3ytv5pucozwakp...,FRAX+FPI (0xf861…),7.31,86,1.858283e+07,1.358405e+08,27
348,Gauge Weight for Week of 15th Sep 2022,2022-09-15 00:00:38,2022-09-20 00:00:38,15535777,closed,bafkreihoebae4qsky5c4da7nljigggd3ytv5pucozwakp...,CRV+cvxCRV (0x9D04…),10.81,39,1.858283e+07,2.008804e+08,27
346,Gauge Weight for Week of 15th Sep 2022,2022-09-15 00:00:38,2022-09-20 00:00:38,15535777,closed,bafkreihoebae4qsky5c4da7nljigggd3ytv5pucozwakp...,FRAX+3Crv (0xd632…),21.78,31,1.858283e+07,4.047341e+08,27


In [54]:
# TODO: For round 27, why do the percentage allocations not add up to 1? 
# TODO: For the earlier voting rounds (before 4), the names used are not consistent with those used in rounds after 4. Normalize these names via tokenization logic. 
df.head()

(
    alt.Chart(
        df[['round', 'fraction', 'choice']].loc[df['round'] >= 4]
    )
    .mark_area()
    .encode(
        x="round:O", 
        y="fraction:Q", 
        color="choice:N",
        tooltip="fraction:Q"
    )
)

/opt/miniconda3/envs/frax-bribes/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

### Votium Bribes 

In [63]:
ddf(df_epoches)
# 4263d2060e5a4cf6fcdac651a85f68b94410e4291ba5168931d1d850c89e377769aa
len('4263d2060e5a4cf6fcdac651a85f68b94410e4291ba5168931d1d850c89e377769aa')

,epoches_id,epoches_initiatedAt,date,round
0,0xc841db892a58168d21262eb8e2f97d651fb354896fa90ac3d6bcfad00319c535,2021-09-16 00:04:43,2021-09-16,1
1,0xe11de27c199ddee0966ed8b24e1ee34bd3363f0fe09fe85510a4fe27914a1027,2021-09-30 00:02:12,2021-09-30,2
2,0x20b26be17c62c2d155f9f9a9a3dac7bc3fcaa48453224bee5de30e238ed32ba2,2021-10-14 00:04:43,2021-10-14,3
3,0xb439c625f9f4929f6b486f79d5ae6850971d179a088721924b53ecc518e33159,2021-10-28 11:02:59,2021-10-28,4
4,0xb59c1e06f38e5daaaa51e672174f6a4a65cf654d1e363ed25bf11153876fbaec,2021-11-08 15:32:47,2021-11-08,5
5,0x77cebe9bd4bc9836f115774c7387839e77abe3dda61370c01d401ed9aa6cdf12,2021-11-11 00:04:34,2021-11-11,6
6,0x0c0550515f038293f31eb10dc002881d1f7f5c170bca3e9a23eec7900d499bf7,2021-11-25 00:03:59,2021-11-25,7
7,0xc26deaa05f45f3f6ad088cb6603d77cb2e826ff98b69e9a122706a37c8694681,2021-12-09 00:10:50,2021-12-09,8
8,0xee224d8e52bc9240eef248d30fa4b1a525c0f686da237620800ab549d1aba0ab,2021-12-23 00:02:44,2021-12-23,9
9,0xc40fe6bb2dabd84a96d12642ecc73e20dbbacd8d7cf0d77cc1d4455e86fce47d,2022-01-06 00:04:07,2022-01-06,10


68

In [58]:
# https://github.com/convex-community/convex-subgraph/blob/main/subgraphs/votium/src/mapping.ts
ADDRESS_FXS = '0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0'.lower()

# Addresses associated with the frax protocol used for votium bribes 
# TODO: Frax controls some subset of the TVL in it's liquidity pools. Need to be cognizant of this because it leads 
#       to a rebate the lowers the cost of bribing. 
# TODO: Frax's vlCVX is not custodied in the investor custodian wallet. 
frax_bribe_addresses = [
    # ('comptroller', '0xb1748c79709f4ba2dd82834b8c82d4a505003f27'),
    ('investor custodian', '0x5180db0237291A6449DdA9ed33aD90a38787621c'),
    # ('cvx locker amo', '0x7038c406e7e2c9f81571557190d26704bb39b8f3'),
    ('frax1.eth', '0x234D953a9404Bf9DbC3b526271d440cD2870bCd2'),
]
# TODO: Check to see if there are bribes in FRAX 
bribes = sg_votium.Query.bribes(first=100000, where={
    "epoch_in": df_epoches.id.unique().tolist(), "token": ADDRESS_FXS
})
df_bribes = sg.query_df([bribes.id, bribes.amount, bribes.choiceIndex, bribes.epoch], pagination_strategy=ShallowStrategy)
print(len(df_bribes))
df_bribes = remove_prefix(df_bribes, "bribes_")
df_bribes['tx_hash'] = df_bribes['id'].apply(lambda _id: _id.split('-')[0])
df_bribes.amount /= 1e18
df_bribes = df_bribes.drop(columns=['id'])
df_bribes = df_bribes.rename(columns={'amount': 'bribe_fxs', 'choiceIndex': 'choice_index'})
ddf(df_bribes.head())

AttributeError: 'DataFrame' object has no attribute 'id'

In [ ]:
# Determine the "from" address for the transaction that contained the bribe 


tx_hashes = df_bribes.tx_hash.unique()

def get_tx(tx_hash): 
    return w3.eth.get_transaction(tx_hash)['from'] 

tx_from_map = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    futures = {tx_hash: executor.submit(get_tx, tx_hash) for tx_hash in tx_hashes}
    for tx_hash, future in futures.items(): 
        tx_from_map[tx_hash] = future.result()

In [ ]:
df_bribes['from'] = df_bribes['tx_hash'].apply(lambda tx_hash: tx_from_map[tx_hash])
address_labels = {addr.lower(): name for name, addr in [
    ('investor custodian', '0x5180db0237291A6449DdA9ed33aD90a38787621c'),
    ('frax1.eth', '0x234D953a9404Bf9DbC3b526271d440cD2870bCd2'),
    # These were in Seba's query but there weren't any bribes associated with them. 
    # ('comptroller', '0xb1748c79709f4ba2dd82834b8c82d4a505003f27'),
    # ('cvx locker amo', '0x7038c406e7e2c9f81571557190d26704bb39b8f3'),
]}
df_bribes['briber_label'] = df_bribes['from'].apply(lambda a: address_labels.get(a.lower()))
label_counts = df_bribes.briber_label.value_counts(dropna=False)
print("Label count for bribing address") 
print(label_counts)
print(f"Dropping {label_counts[None]} bribe(s)")
df_bribes = df_bribes.loc[~df_bribes.briber_label.isna()] 
df_bribes = df_bribes.drop(columns=['tx_hash', 'from']).sort_values(['briber_label', 'choice_index']).reset_index(drop=True)

df_bribes.head()

In [ ]:
df_epoches.head()

In [ ]:
df.head()

In [ ]:
df_votium = df_bribes.merge(df_epoches, how='left', left_on='epoch_id', right_on='epoches_id', validate='m:1')
df_votium = (
    df_votium[['date', 'round', 'choice_index', 'bribe_fxs', 'epoches_initiatedAt', 'epoch_bribeCount', 'briber_label', ]]
    .sort_values(['epoches_initiatedAt', 'round', 'choice_index'])
    .reset_index(drop=True)
) 
ddf(df_votium)